In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

Import Model from tf_hub

In [2]:
model = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = model.signatures['serving_default']

Metal device set to: Apple M1 Pro


2022-06-24 08:43:35.086929: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-24 08:43:35.087087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [4]:
# Draw Landmarks
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [5]:
# Edges with corresponding Colors
EDGES = {
    (0, 1): (255,0,0),
    (0, 2): (0,255,0),
    (1, 3): (0,0,255),
    (2, 4): (255,255,0),
    (0, 5): (255,0,255),
    (0, 6): (0,255,255),
    (5, 7): (0,0,0),
    (7, 9): (255,255,255),
    (6, 8): (255,0,0),
    (8, 10): (0,255,0),
    (5, 6): (0,0,255),
    (5, 11): (255,255,0),
    (6, 12): (255,0,255),
    (11, 12): (0,255,255),
    (11, 13): (0,0,0),
    (13, 15): (255,255,255),
    (12, 14): (255,0,0),
    (14, 16): (0,255,0)
}

In [6]:
# Draw Lines between Landmarks
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)

Webcam Capture (Hold 'q' (or press 3 times) to stop capture, close window and finish cell)

In [7]:
cap = cv2.VideoCapture(0)
#cap.set(3,1920) # adjust width
#cap.set(4,1080) # adjust height


while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('Movenet Multipose', frame)
    
    # Hold or Press 3 Times to finish
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
        
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

2022-06-24 08:43:56.272013: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-24 08:43:56.274136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
